# Extracción de Tablas desde PDF usando OCR

Este notebook demuestra diferentes estrategias para extraer tablas de archivos PDF usando OCR:

1. Usando pytesseract + pdf2image (OCR básico)
2. Usando tabula-py (para PDFs con tablas bien estructuradas)
3. Usando camelot (para casos más complejos)

Primero, instalemos las dependencias necesarias:


In [218]:
import pdfplumber
import pandas as pd

# Ruta al archivo PDF
pdf_file = "/Users/mwolinsky/Downloads/COTZ0000100035098-Policuyo.pdf"  # Reemplaza con la ruta a tu PDF

with pdfplumber.open(pdf_file) as pdf:
    # Asumiendo que la tabla principal está en la primera página con datos
    for page in pdf.pages:
        tables = page.extract_tables()
        if tables:
            raw_table = tables[0]
            break

# Pasar a DataFrame pero sin preocuparnos de encabezados todavía
df_raw = pd.DataFrame(raw_table)

# Usualmente la primera fila es el encabezado real
header = df_raw.iloc[0].tolist()
df_data = df_raw.iloc[1:].reset_index(drop=True)

# Cada columna es un string largo con "\n"
lists = []
for col in range(df_data.shape[1]):
    # Extraer la columna y transformarla en lista
    values = '\n'.join(df_data.iloc[:, col].values)  # por si son varias filas
    values_list = [x.strip() for x in values.strip().split('\n') if x.strip() != '']
    lists.append(values_list)

# Obtener la cantidad máxima de filas (suele ser la columna más larga)
n_rows = max(len(l) for l in lists)

# Rellenar listas cortas
lists = [l + ['']*(n_rows-len(l)) for l in lists]

# Construir DataFrame final
df_final = pd.DataFrame({header[i]: lists[i] for i in range(len(header))})


In [219]:
import re
text = ""
with pdfplumber.open(pdf_file) as pdf:
    for page in pdf.pages:
        text += page.extract_text() + "\n"

# Patrón regex para encontrar los totales
pattern = r'(Subtotal Cotización|Bonificación|Subtotal Neto|IVA|Total Cotización)\s*:\s*([\d.,]+)'

matches = re.findall(pattern, text)

# Convertimos los resultados en diccionario
resultados = {label: float(valor.replace(',', '').replace('.', '', valor.count('.')-1)) for label, valor in matches}

resultados_df = pd.DataFrame([resultados])

resultados_df["PCT_Bonificacion"] = resultados_df["Bonificación"] / resultados_df["Subtotal Cotización"]

In [220]:
resultados_df

,Subtotal Cotización,Bonificación,Subtotal Neto,IVA,Total Cotización,PCT_Bonificacion
0,3695.94,1293.58,2402.36,442.58,2844.94,0.35


In [221]:
df_final.dtypes

Descripción Artículo    object
Desc. Adicional         object
Cantidad                object
Uni                     object
Precio Unit             object
% IVA                   object
% Desc.                 object
Importe                 object
dtype: object

In [222]:
df_final["Cantidad"] = pd.to_numeric(df_final["Cantidad"], errors="coerce")
df_final["Precio_Unit"] = pd.to_numeric(df_final["Precio Unit"], errors="coerce")

In [224]:
# Asegurarse de que "% Desc." y "% IVA" sean numéricos antes de operar
df_final["% Desc."] = pd.to_numeric(df_final["% Desc."], errors="coerce").fillna(0)
df_final["% IVA"] = pd.to_numeric(df_final["% IVA"], errors="coerce").fillna(0)
df_final["Importe"] = pd.to_numeric(df_final["Importe"], errors="coerce").fillna(0)

pct_bonificacion = pd.to_numeric(resultados_df["PCT_Bonificacion"].iloc[0], errors="coerce")
if pd.isna(pct_bonificacion):
    pct_bonificacion = 0

df_final["Precio_Lista"] = (
    df_final["Cantidad"]
    * df_final["Precio_Unit"]
)

df_final["Precio Lista con Descuento Prod."] = df_final["Precio_Lista"] * (1 - df_final["% Desc."] / 100)
df_final["Precio Neto"] = df_final["Precio Lista con Descuento Prod."] * (1 - pct_bonificacion)
df_final["Precio Neto Unitario"] = df_final["Precio Neto"] / df_final["Cantidad"]
df_final["Precio con Impuestos"] = df_final["Precio Neto"] * (1 + df_final["% IVA"] / 100)

In [226]:
df_final["Precio"] = df_final["Precio Neto Unitario"]

In [208]:
df_final


,Descripción Artículo,Desc. Adicional,Cantidad,Uni,Precio Unit,% IVA,% Desc.,Importe,Precio_Unit,Precio_Lista,Precio Lista con Descuento Prod.,Precio Neto
0,Adhesivo PVC extra refor 473cc,WELD ON 717,120.0,UND,19.02923,21.0,10.0,2055.16,19.02923,2283.5076,2055.156840,1335.851390
1,Buje Reduccion 25*20,ERA,120.0,UND,0.04820,21.0,25.0,4.34,0.04820,5.7840,4.338000,2.819699
2,Codo 45° 25mm PN10,ERA,50.0,UND,0.17360,21.0,25.0,6.51,0.17360,8.6800,6.510000,4.231498
3,Codo 90° 75mm PN10,ERA,50.0,UND,3.63020,21.0,25.0,136.13,3.63020,181.5100,136.132500,88.486088
4,Codo 90° 90mm PN10,ERA,20.0,UND,6.31550,21.0,25.0,94.73,6.31550,126.3100,94.732500,61.576099
5,Cupla 110mm JP PN10,ERA,60.0,UND,5.38990,21.0,25.0,242.55,5.38990,323.3940,242.545500,157.654509
6,Cupla 20mm JP PN10,ERA,300.0,UND,0.11090,21.0,25.0,24.95,0.11090,33.2700,24.952500,16.219118
7,Cupla 25mm JP PN10,ERA,300.0,UND,0.16390,21.0,25.0,36.88,0.16390,49.1700,36.877500,23.970365
8,Cupla 32mm JP PN10,ERA,50.0,UND,0.31820,21.0,25.0,11.93,0.31820,15.9100,11.932500,7.756122
9,"Cupla 50* 1 1/2""",ERA,72.0,UND,0.84850,21.0,25.0,45.82,0.84850,61.0920,45.819000,29.782338


In [235]:
# En vez de agregar filas de totales al mismo DataFrame principal, creamos un DataFrame separado para los totales/resúmenes

# Calcular los totales/resúmenes y redondear a 2 decimales
subtotal = round(pd.to_numeric(df_final["Precio Neto"], errors="coerce").sum(), 2)

precio_neto_21 = pd.to_numeric(df_final.loc[df_final["% IVA"] == 21, "Precio Neto"].replace("", 0), errors="coerce").fillna(0)
iva_21 = round((precio_neto_21 * 0.21).sum(), 2)

precio_neto_105 = pd.to_numeric(df_final.loc[df_final["% IVA"] == 10.5, "Precio Neto"].replace("", 0), errors="coerce").fillna(0)
iva_105 = round((precio_neto_105 * 0.105).sum(), 2)

total = round(pd.to_numeric(df_final["Precio con Impuestos"], errors="coerce").sum(), 2)

# Crear un DataFrame resumen
resumen_df = pd.DataFrame({
    "Concepto": ["Subtotal", "IVA 21%", "IVA 10.5%", "Total"],
    "Importe": [subtotal, iva_21, iva_105, total]
})

# Mostrar el DataFrame de resumen
resumen_df


,Concepto,Importe
0,Subtotal,2388.53
1,IVA 21%,377.75
2,IVA 10.5%,61.92
3,Total,2828.20


In [236]:
df_result = df_final[["Descripción Artículo", "Desc. Adicional", "Cantidad", "Precio", "% IVA", "Precio Neto"]].copy()
df_result["Cantidad"] = df_result["Cantidad"].round(2)
df_result["Precio"] = df_result["Precio"].round(2)
df_result["% IVA"] = df_result["% IVA"].round(2)
df_result["Precio Neto"] = df_result["Precio Neto"].round(2)

In [237]:
import pandas as pd

# Crear un ExcelWriter
with pd.ExcelWriter("resultado.xlsx", engine="xlsxwriter") as writer:
    # Escribir df_final en la hoja "Hoja1"
    df_result.to_excel(writer, index=False, sheet_name="Hoja1", startrow=0)
    
    # Obtener el número de filas de df_final
    n_filas = len(df_final)
    
    # Escribir resumen_df dos filas debajo de df_final
    resumen_df.to_excel(writer, index=False, sheet_name="Hoja1", startrow=n_filas + 2)
